<center>

**Vježba 3 - MLP klasifikator**

</center>

U ovoj vježbi želimo vidjeti kako učiti MLP (*multilayered perceptron*) klasifikator na temelju realnog skupa podataka. U tu će se svrhu koristiti *breast cancer* skup podataka koji ima binarne labele (0 - nije, 1 - jest). Imajmo na umu da je ovaj skup podataka već dovoljno pročišćen da se može koristiti u ovakvom originalnom stanju bez prevelikih ulaznih čišćenja i transformacija.  
[Breast cancer dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset)

Prvo učitamo pyton module koje ćemo koristiti. Učitavamo MLP klasifikator. Ujedno učitavamo i *joblib* modul kako bismo pokazali spremanje i učitavanje modela.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import pandas as pd
from joblib import dump, load

Prvo pogledajmo sadržaj skupa podataka *breast cancer*. Učitamo skup podataka u kompleksnu strukturu ```bcds```.
Parametri poziva funkcije za učitavanje skupa podataka su:
- ```return_X_y=False``` - ne želimo automatsko razdvajanje ciljne varijable
- ```as_frame=True``` - želimo da nam vraćeni skup podataka sadrži pandas ```DataFrame```. Taj se *dataframe* nalazi u svojstvu ```bcds.data```

Na kraju pozivamo metodu ```describe()``` na pandas *dataframe-u* koja vraća osnovnu statističku obradu tog skupa podataka kao: minimalnu vrijednosti, standardnu devijaciju, maksimalnu vrijednost, brojeve i slično.

Proučite podatke. Primijetite da su podaci nisu anonimizirani, a nazivi značajki su prisutni.

In [ ]:
bcds = load_breast_cancer(return_X_y=False, as_frame=True)

display(bcds.feature_names)
display(bcds.data)
display(bcds.data.describe().transpose())

Odradimo učitavanje skupa podataka tako da posebno učitamo ciljnu labelu. Ciljna varijabla klasificira podatke u dvije klase.

Zatim razdvojimo naš skup podataka na podskup za učenje i testiranje. Ispišimo neke osnovne elemente podskupa podataka, te pogledajmo kako izgleda polje labela koje ćemo koristiti u procesu učenja.

In [ ]:
bc_X, bc_y = load_breast_cancer(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(bc_X, bc_y)

display(X_train.shape)
display(y_train.shape)
display(y_train)

Odradimo skaliranje podataka. Neki klasifikatori su osjetljivi na neskalirane podatke. O ovome će biti riječ u narednim predavanjima.

U ovom slučaju koristimo standardno skaliranje podataka.  
[StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

Standardizirano skaliranje je svođenje vrijednosti na $x_{s_i} = (x_i - \mu_i) / \sigma_i$. Od vrijednosti oduzmemo srednju vrijednost te značajke i tu razliku podijelimo sa standardnom devijacijom te značajke.

Na kraju skaliranja ponovno ispisujemo podskupove za učenje i testiranje.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

display(pd.DataFrame(X_train, columns=bc_X.columns))
display(pd.DataFrame(X_train, columns=bc_X.columns).describe().transpose())

Definiramo MLP klasifikator koji ima dva sloja od 30 skrivenih neurona. Koristimo *Adam* optimizaciju (adaptivna optimizacija). Tražimo da nam kod učenja MLP klasifikator uzima podatke onim redom kako su u skupu za učenje, kako bismo uvijek dobili isti rezultat.  
[Adam](https://optimization.cbe.cornell.edu/index.php?title=Adam)


Probajte mijenjati ove parametre MLP klasifikatora.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(30,30), solver='adam', verbose=True, random_state=0)
mlp.fit(X_train,y_train)

Zatim testiramo naučeni model korištenjem testnog podskupa podataka. Ispisujemo matricu zabune (*confusion matrix*) s kojom vidimo kombinacije točnih i krivih klasifikacije. Nakon toga ispisujemo i detaljniji izvještaj klasifikacije.

Koncentrirajte se na točnost klasifikacije.

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred = mlp.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

Pogledajmo kakve su težinske matrice MLP klasifikatora.

Ponovno, ova lista ovisi o strukturi MLP-a.

In [ ]:
print("Weights between input and first hidden layer:")
print(mlp.coefs_[0])
print("Weights between first hidden and second hidden layer:")
print(mlp.coefs_[1])
print("Weights between second hidden and output layer:")
print(mlp.coefs_[2])

Zatim spremimo taj klasifikator u *joblib* datoteku. Treba primijetiti da je *joblib* binarna datoteka i da su detalji toga što se sprema skriveni. No, svakako su to hipeparametri neuronske mreže koju smo prethodno naučili raspoznavati podatke za učenje.

Zatim obrišemo objekt klasifikatora, kako bismo pokazali da će svaka buduća klasifikacija biti temeljena na podacima spremljenima u *joblib* datoteku.

In [ ]:
dump(mlp, 'mlp_breastcancer.joblib')
del mlp

Ponovno učitamo klasifikator iz *joblib* datoteke i ponovimo klasifikaciju sa testnim podskupom.

In [ ]:
mlp2 = load('mlp_breastcancer.joblib')

y_pred2 = mlp2.predict(X_test)

print(confusion_matrix(y_test,y_pred2))
print(classification_report(y_test,y_pred2))

Na ovaj se način modeli spremaju u repozitorij modela i prenose na poslužitelje modela.